# Incremental Learning Model
This origin comming from https://www.kaggle.com/code/lblhandsome/optiver-robust-best-single-model/notebook

# Global params

In [1]:
from pathlib import Path
import os
import warnings
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt
import tensorflow as tf
import gc  # Garbage collection for memory management
import os  # Operating system-related functions
import time  # Time-related functions
import warnings  # Handling warnings
from itertools import combinations  # For creating combinations of elements
from warnings import simplefilter  # Simplifying warning handling
import joblib  # For saving and loading models
import numpy as np  # Numerical operations
import pandas as pd  # Data manipulation and analysis
from sklearn.metrics import mean_absolute_error  # Metric for evaluation
from sklearn.model_selection import KFold, TimeSeriesSplit  # Cross-validation techniques

# Disable warnings to keep the code clean
warnings.filterwarnings("ignore")
simplefilter(action="ignore", category=pd.errors.PerformanceWarning)

warnings.filterwarnings("ignore")

seed = 2023
DATA_COUNT_IN_SAME_BUCKET = 55 # 同じbucket内のデータ数

# valid_split_day = 435  # Split day for time series data

# For kaggle environment
if os.environ.get("KAGGLE_DATA_PROXY_TOKEN") != None:
    BASE_OUTPUT_PATH = Path(f'/kaggle/working')
    BASE_INPUT_PATH = Path(f'/kaggle/input/optiver-trading-at-the-close')
    TRAIN_FILE = Path(f'{BASE_INPUT_PATH}/train.csv')
    TEST_FILE = Path(f'{BASE_INPUT_PATH}/example_test_files/test.csv')
    
    IS_LOCAL = False # If kaggle environment, set False
    IS_INFER = True # If kaggle environment, set True
    IS_USE_SAVED_MODEL = False # Use saved model or not
    IS_MIN_LEARN = False # Use min learning or not
    USE_OPTUNA = False # Use optuna or not
    USE_CONTINUOUS_UPDATE = True # Use test date on train or not
    USE_ALL_FEATUTES = True # Use all features or not
    USE_REVEALED_TARGETS = True # Use revealed targets or not
    USE_INDEX = True # Use index or not
    IS_DEBUG = False

    SAMPLE_SUBMISSION_FILE = Path(f'{BASE_INPUT_PATH}/example_test_files/sample_submission.csv')
    REVEALED_TARGETS_FILE = Path(f'{BASE_INPUT_PATH}/example_test_files/revealed_targets.csv')

    stopping_rounds = 100 # early_stopping用コールバック関数
    num_boost_round = 1000 # 計算回数
    update_num_boost_round = 1000 # 再学習の計算回数
    num_folds = 3 # クロスバリデーションの分割数
    continuos_dataset_span = 3 # DATA_COUNT_IN_SAME_BUCKET * continuos_dataset_span が更新の使用する対象のデータ
    continuos_train_span = 1 # DATA_COUNT_IN_SAME_BUCKET * continuos_train_span が更新の頻度

    DEVICE = 'gpu' # cpu or gpu
    OPTUNA_TIME_BUDGET = 60 * 60 * 4 # 1 hours
    TARGET_STOCK_IDS = [0, 1]

    optuna_params = {
        'boosting_type': 'gbdt',
        'objective': 'regression',         # 回帰
        'metric': 'rmse',                  # 損失（誤差）
        'verbosity': -1,
        'deterministic':True, #再現性確保用のパラメータ
        'force_row_wise':True,  #再現性確保用のパラメータ
        'device': DEVICE
    }

    """
    lgb_params = {
        'boosting_type': 'gbdt',
        'objective': 'regression', 
        'metric': 'rmse', 
        'verbosity': -1, 
        'device': DEVICE,
        'feature_pre_filter': False, 
        'lambda_l1': 0.0,
        'lambda_l2': 0.0,
        'num_leaves': 31, 
        'feature_fraction': 0.8, 
        'bagging_fraction': 1.0, 
        'bagging_freq': 0, 
        'min_child_samples': 20
    }
    """
   
    lgb_params = {
        'task': 'train',                   # 学習
        'objective': 'regression',                # 目的関数の種類。ここでは回帰タスクを指定
        'metric': 'rmse',                          # 評価指標
        'boosting_type': 'gbdt',                  # ブースティングタイプ。勾配ブースティング決定木
        "n_estimators": 32,                        # ブースティングに使用する木の数。多いほど性能が向上するが計算コストが増加
        "num_leaves": 64,                         # 木に存在する最大の葉の数。大きい値は精度を向上させるが過学習のリスクが増加
        "subsample": 0.8,                         # 各木のトレーニングに使用されるデータの割合。過学習を防ぐために一部のデータをサンプリング
        "colsample_bytree": 0.8,                  # 木を構築する際に使用される特徴の割合。特徴のサブセットを使用し過学習を防ぐ
        "learning_rate": 0.01,                 # 学習率。小さい値は堅牢なモデルを生成するが収束に時間がかかる
        'max_depth': 32,                           # 木の最大の深さ。深い木は複雑なモデルを作成するが過学習のリスクがある
        "device": DEVICE,                         # トレーニングに使用するデバイス（CPUまたはGPU）
        "verbosity": -1,                          # LightGBMのログ出力のレベル。-1はログを出力しないことを意味する
       # "importance_type": "gain",                # 特徴重要度を計算する際の指標。"gain"は分割による平均情報利得
        'lambda_l1': 0.5,                         # L1正則化項の係数。過学習を防ぐためにモデルの複雑さにペナルティを課す
        'lambda_l2': 0.5,                         # L2正則化項の係数。同じく過学習を防ぐ
        'bagging_freq': 5,                 # バギング実施頻度
        'min_child_samples': 10,           # 葉に含まれる最小データ数
        'seed': seed,                       # シード値
    }

# For local environment
else:
    BASE_OUTPUT_PATH = Path(f'../output')
    BASE_INPUT_PATH = Path(f'../kaggle/input/optiver-trading-at-the-close')
    TRAIN_FILE = Path(f'{BASE_INPUT_PATH}/train.csv')
    TEST_FILE = Path(f'{BASE_INPUT_PATH}/test.csv')

    SAMPLE_SUBMISSION_FILE = Path(f'{BASE_INPUT_PATH}/sample_submission.csv')
    REVEALED_TARGETS_FILE = Path(f'{BASE_INPUT_PATH}/revealed_targets.csv')

    IS_LOCAL = True
    IS_INFER = True
    IS_USE_SAVED_MODEL = False # Use saved model or not
    IS_MIN_LEARN = True
    USE_OPTUNA = False # Use optuna or not
    USE_ALL_FEATUTES = True # Use all features or not
    USE_CONTINUOUS_UPDATE = True # Use test date on train or not
    USE_REVEALED_TARGETS = False # Use revealed targets or not 
    USE_INDEX = True # Use index or not
    IS_DEBUG = True
    TARGET_STOCK_IDS = [0]

    # For training
    stopping_rounds = 1 # early_stopping用コールバック関数
    num_boost_round = 1 # 計算回数
    update_num_boost_round = 1
    num_folds = 2 # クロスバリデーションの分割数
    continuos_dataset_span = 3 # DATA_COUNT_IN_SAME_BUCKET * continuos_dataset_span が更新の使用する対象のデータ
    continuos_train_span = 2 # DATA_COUNT_IN_SAME_BUCKET * continuos_train_span が更新の頻度

    DEVICE = 'cpu' # cpu or gpu
    OPTUNA_TIME_BUDGET = 60 # 1 min

    optuna_params = {
        'boosting_type': 'gbdt',
        'objective': 'regression',         # 回帰
        'metric': 'rmse',                  # 損失（誤差）
        'verbosity': -1,
        'deterministic':True, #再現性確保用のパラメータ
        'force_row_wise':True,  #再現性確保用のパラメータ
        'device': DEVICE
    }

    lgb_params = {
        'task': 'train',                   # 学習
        'boosting_type': 'gbdt',           # GBDT
        'objective': 'regression',         # 回帰
        'metric': 'rmse',                  # 損失（誤差）
        'learning_rate': 0.01,             # 学習率
        'lambda_l1': 0.5,                  # L1正則化項の係数
        'lambda_l2': 0.5,                  # L2正則化項の係数
        'num_leaves': 10,                  # 最大葉枚数
        'feature_fraction': 0.5,           # ランダムに抽出される列の割合
        'bagging_fraction': 0.5,           # ランダムに抽出される標本の割合
        'bagging_freq': 5,                 # バギング実施頻度
        'min_child_samples': 10,           # 葉に含まれる最小データ数
        'seed': seed,                       # シード値
        "device": DEVICE,
        'verbosity': -1
    }


print(f"BASE_OUTPUT_PATH: {BASE_OUTPUT_PATH}")
print(f"BASE_INPUT_PATH: {BASE_INPUT_PATH}")
print(f"TRAIN_FILE: {TRAIN_FILE}")
print(f"TEST_FILE: {TEST_FILE}")
print(f"IS_LOCAL: {IS_LOCAL}")
print(f"IS_INFER: {IS_INFER}")
print(f"IS_USE_SAVED_MODEL: {IS_USE_SAVED_MODEL}")
print(f"IS_MIN_LEARN: {IS_MIN_LEARN}")
print(f"USE_OPTUNA: {USE_OPTUNA}")
print(f"USE_CONTINUOUS_UPDATE: {USE_CONTINUOUS_UPDATE}")
print(f"USE_ALL_FEATUTES: {USE_ALL_FEATUTES}")
print(f"USE_REVEALED_TARGETS: {USE_REVEALED_TARGETS}")
print(f"USE_INDEX: {USE_INDEX}")


BASE_OUTPUT_PATH: ../output
BASE_INPUT_PATH: ../kaggle/input/optiver-trading-at-the-close
TRAIN_FILE: ../kaggle/input/optiver-trading-at-the-close/train.csv
TEST_FILE: ../kaggle/input/optiver-trading-at-the-close/test.csv
IS_LOCAL: True
IS_INFER: True
IS_USE_SAVED_MODEL: False
IS_MIN_LEARN: True
USE_OPTUNA: False
USE_CONTINUOUS_UPDATE: True
USE_ALL_FEATUTES: True
USE_REVEALED_TARGETS: False
USE_INDEX: True


# Functions

## Memory Functions

In [2]:
%%time 

from gc import collect;
from psutil import Process;
from os import system, getpid, walk;

# Defining global configurations and functions:-

    
def GetMemUsage():
    """
    This function defines the memory usage across the kernel. 
    Source-
    https://stackoverflow.com/questions/61366458/how-to-find-memory-usage-of-kaggle-notebook
    """;
    
    pid = getpid();
    py = Process(pid);
    memory_use = py.memory_info()[0] / 2. ** 30;
    return f"RAM memory GB usage = {memory_use :.4}";


collect();
print(GetMemUsage())

RAM memory GB usage = 0.4249
CPU times: user 75.7 ms, sys: 1.86 ms, total: 77.5 ms
Wall time: 77.1 ms


In [3]:
# 🧹 Function to reduce memory usage of a Pandas DataFrame
def reduce_mem_usage(df, name: str):
    """
    Iterate through all numeric columns of a dataframe and modify the data type
    to reduce memory usage.
    """
    
    # 📏 Calculate the initial memory usage of the DataFrame
    start_mem = df.memory_usage().sum() / 1024**2

    # 🔄 Iterate through each column in the DataFrame
    for col in df.columns:
        col_type = df[col].dtype

        # Check if the column's data type is not 'object' (i.e., numeric)
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            
            # Check if the column's data type is an integer
            if str(col_type)[:3] == "int":
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                # Check if the column's data type is a float
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float32)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float32)


    print(f"Memory usage of {name} is {start_mem:.2f} MB")
    end_mem = df.memory_usage().sum() / 1024**2
    print(f"Memory usage after optimization is: {end_mem:.2f} MB")
    decrease = 100 * (start_mem - end_mem) / start_mem
    print(f"Decreased by {decrease:.2f}%")

    # 🔄 Return the DataFrame with optimized memory usage

    start_mem = df.memory_usage().sum() / 1024**2
    return df

## API Function

In [4]:
%%time 

from typing import Sequence, Tuple
import pandas as pd

# for local execution
class MockApi:
    def __init__(self):
        '''
        YOU MUST UPDATE THE FIRST THREE LINES of this method.
        They've been intentionally left in an invalid state.

        Variables to set:
            input_paths: a list of two or more paths to the csv files to be served
            group_id_column: the column that identifies which groups of rows the API should serve.
                A call to iter_test serves all rows of all dataframes with the current group ID value.
            export_group_id_column: if true, the dataframes iter_test serves will include the group_id_column values.
        '''
        self.input_paths: Sequence[str] = [TEST_FILE, REVEALED_TARGETS_FILE, SAMPLE_SUBMISSION_FILE]
        self.group_id_column: str = 'time_id'
        self.export_group_id_column: bool = True
        # iter_test is only designed to support at least two dataframes, such as test and sample_submission
        assert len(self.input_paths) >= 2

        self._status = 'initialized'
        self.predictions = []

    def iter_test(self) -> Tuple[pd.DataFrame]:
        '''
        Loads all of the dataframes specified in self.input_paths,
        then yields all rows in those dataframes that equal the current self.group_id_column value.
        '''
        if self._status != 'initialized':

            raise Exception('WARNING: the real API can only iterate over `iter_test()` once.')

        dataframes = []
        for pth in self.input_paths:
            dataframes.append(pd.read_csv(pth, low_memory=False))
        group_order = dataframes[0][self.group_id_column].drop_duplicates().tolist()
        dataframes = [df.set_index(self.group_id_column) for df in dataframes]

        for group_id in group_order:
            self._status = 'prediction_needed'
            current_data = []
            for df in dataframes:
                cur_df = df.loc[group_id].copy()
                # returning single line dataframes from df.loc requires special handling
                if not isinstance(cur_df, pd.DataFrame):
                    cur_df = pd.DataFrame({a: b for a, b in zip(cur_df.index.values, cur_df.values)}, index=[group_id])
                    cur_df.index.name = self.group_id_column
                cur_df = cur_df.reset_index(drop=not(self.export_group_id_column))
                current_data.append(cur_df)
            yield tuple(current_data)

            while self._status != 'prediction_received':
                print('You must call `predict()` successfully before you can continue with `iter_test()`', flush=True)
                yield None

        with open('submission.csv', 'w') as f_open:
            pd.concat(self.predictions).to_csv(f_open, index=False)
        self._status = 'finished'

    def predict(self, user_predictions: pd.DataFrame):
        '''
        Accepts and stores the user's predictions and unlocks iter_test once that is done
        '''
        if self._status == 'finished':
            raise Exception('You have already made predictions for the full test set.')
        if self._status != 'prediction_needed':
            raise Exception('You must get the next test sample from `iter_test()` first.')
        if not isinstance(user_predictions, pd.DataFrame):
            raise Exception('You must provide a DataFrame.')

        self.predictions.append(user_predictions)
        self._status = 'prediction_received'

def make_env():
    return MockApi()

CPU times: user 44 µs, sys: 1 µs, total: 45 µs
Wall time: 47 µs


## Pandas Functions

In [5]:
def pd_display_max():
    pd.set_option('display.max_rows', None)  # 行の最大表示数を無制限に設定
    pd.set_option('display.max_columns', None)  # 列の最大表示数を無制限に設定
    pd.set_option('display.width', None)  # 表示幅を拡張
    pd.set_option('display.max_colwidth', None)  # 列の幅を最大に設定

def pd_clear_display_max():
    pd.set_option('display.max_rows', 10)
    pd.set_option('display.max_columns', 10)
    pd.set_option('display.width', None)  # 表示幅を拡張
    pd.set_option('display.max_colwidth', None)  # 列の幅を最大に設定

## Feature Functions

In [6]:
# 🏎️ Import Numba for just-in-time (JIT) compilation and parallel processing
from numba import njit, prange

# 📊 Function to compute triplet imbalance in parallel using Numba
@njit(parallel=True)
def compute_triplet_imbalance(df_values, comb_indices):
    num_rows = df_values.shape[0]
    num_combinations = len(comb_indices)
    imbalance_features = np.empty((num_rows, num_combinations))

    # 🔁 Loop through all combinations of triplets
    for i in prange(num_combinations):
        a, b, c = comb_indices[i]
        
        # 🔁 Loop through rows of the DataFrame
        for j in range(num_rows):
            max_val = max(df_values[j, a], df_values[j, b], df_values[j, c])
            min_val = min(df_values[j, a], df_values[j, b], df_values[j, c])
            mid_val = df_values[j, a] + df_values[j, b] + df_values[j, c] - min_val - max_val
            
            # 🚫 Prevent division by zero
            if mid_val == min_val:
                imbalance_features[j, i] = np.nan
            else:
                imbalance_features[j, i] = (max_val - mid_val) / (mid_val - min_val)

    return imbalance_features

# 📈 Function to calculate triplet imbalance for given price data and a DataFrame
def calculate_triplet_imbalance_numba(price, df):
    # Convert DataFrame to numpy array for Numba compatibility
    df_values = df[price].values
    comb_indices = [(price.index(a), price.index(b), price.index(c)) for a, b, c in combinations(price, 3)]

    # Calculate the triplet imbalance using the Numba-optimized function
    features_array = compute_triplet_imbalance(df_values, comb_indices)

    # Create a DataFrame from the results
    columns = [f"{a}_{b}_{c}_imb2" for a, b, c in combinations(price, 3)]
    features = pd.DataFrame(features_array, columns=columns)

    return features

In [7]:
# 📊 Function to generate imbalance features
def imbalance_features(df):
    if DEVICE == 'gpu':
        import cudf
        df = cudf.from_pandas(df)
    
    # Define lists of price and size-related column names
    prices = ["reference_price", "far_price", "near_price", "ask_price", "bid_price", "wap"]
    sizes = ["matched_size", "bid_size", "ask_size", "imbalance_size"]

    # V1 features
    # Calculate various features using Pandas eval function
    df["volume"] = df.eval("ask_size + bid_size")
    df["mid_price"] = df.eval("ask_price + bid_price")/2
    df["liquidity_imbalance"] = df.eval("(bid_size-ask_size)/(bid_size+ask_size)")
    df["matched_imbalance"] = df.eval("imbalance_size-matched_size")/df.eval("matched_size+imbalance_size")
    df["size_imbalance"] = df.eval("bid_size / ask_size")
    
    # Create features for pairwise price imbalances
    for c in combinations(prices, 2):
        df[f"{c[0]}_{c[1]}_imb"] = df.eval(f"({c[0]} - {c[1]})/({c[0]} + {c[1]})")
        
    # V2 features
    # Calculate additional features
    df["imbalance_momentum"] = df.groupby(['stock_id'])['imbalance_size'].diff(periods=1) / df['matched_size']
    df["price_spread"] = df["ask_price"] - df["bid_price"]
    df["spread_intensity"] = df.groupby(['stock_id'])['price_spread'].diff()
    df['price_pressure'] = df['imbalance_size'] * (df['ask_price'] - df['bid_price'])
    df['market_urgency'] = df['price_spread'] * df['liquidity_imbalance']
    df['depth_pressure'] = (df['ask_size'] - df['bid_size']) * (df['far_price'] - df['near_price'])
    # Calculate the imbalance ratio
    df['match_balance'] = ( df['matched_size']  + (df['imbalance_buy_sell_flag'] * df['imbalance_size'])) / df['matched_size']
    
    # Calculate various statistical aggregation features
    
        
    # V3 features
    # Calculate shifted and return features for specific columns
    for col in ['matched_size', 'imbalance_size', 'reference_price', 'imbalance_buy_sell_flag']:
        for window in [1, 2, 3, 10]:
            df[f"{col}_shift_{window}"] = df.groupby('stock_id')[col].shift(window)
            df[f"{col}_ret_{window}"] = df.groupby('stock_id')[col].pct_change(window)
    
    # Calculate diff features for specific columns
    for col in ['ask_price', 'bid_price', 'ask_size', 'bid_size']:
        for window in [1, 2, 3, 10]:
            df[f"{col}_diff_{window}"] = df.groupby("stock_id")[col].diff(window)
    if DEVICE == 'gpu':
        df = df.to_pandas()
    # Replace infinite values with 0
    return df.replace([np.inf, -np.inf], 0)

def numba_imb_features(df):
    prices = ["reference_price", "far_price", "near_price", "ask_price", "bid_price", "wap"]
    sizes = ["matched_size", "bid_size", "ask_size", "imbalance_size"]
    
    for func in ["mean", "std", "skew", "kurt"]:
        df[f"all_prices_{func}"] = df[prices].agg(func, axis=1)
        df[f"all_sizes_{func}"] = df[sizes].agg(func, axis=1)
        
    # Calculate triplet imbalance features using the Numba-optimized function
    for c in [['ask_price', 'bid_price', 'wap', 'reference_price'], sizes]:
        triplet_feature = calculate_triplet_imbalance_numba(c, df)
        df[triplet_feature.columns] = triplet_feature.values
    return df

# 📅 Function to generate time and stock-related features
def other_features(df):
    df["dow"] = df["date_id"] % 5  # Day of the week
    df["seconds"] = df["seconds_in_bucket"] % 60  # Seconds
    df["minute"] = df["seconds_in_bucket"] // 60  # Minutes

    # Map global features to the DataFrame
    for key, value in global_stock_id_feats.items():
        df[f"global_{key}"] = df["stock_id"].map(value.to_dict())

    return df

# 🚀 Function to generate all features by combining imbalance and other features
def generate_basic_features(df):
    prev_cols = list(df.columns)

    # Generate imbalance features
    df = imbalance_features(df)
    df = numba_imb_features(df)

    df = reduce_mem_usage(df, "generate_basic_features")

    collect()  # Perform garbage collection to free up memory

    return df

# Generationg train dataset

In [8]:
def load_train_dataset():
    df = pd.read_csv(TRAIN_FILE)
    # 🧹 Remove rows with missing values in the "target" column
    df = df.dropna(subset=["target"])
    # 🔁 Reset the index of the DataFrame and apply the changes in place
    df.reset_index(drop=True, inplace=True)
    return df

def load_test_dataset():
    df_test = pd.read_csv(TEST_FILE)
    
    df_revealed_targets = pd.read_csv(REVEALED_TARGETS_FILE)
    df_revealed_targets = df_revealed_targets.dropna(subset=['date_id', 'seconds_in_bucket', 'stock_id'])
    df_revealed_targets['revealed_date_id'] = df_revealed_targets['revealed_date_id'].astype(int).astype(str)
    df_revealed_targets['seconds_in_bucket'] = df_revealed_targets['seconds_in_bucket'].astype(int).astype(str)
    df_revealed_targets['stock_id'] = df_revealed_targets['stock_id'].astype(int).astype(str)  # Converting to int first to remove any decimal points
    df_revealed_targets['date_id'] = df_revealed_targets['date_id'].astype(int).astype(str)
    df_revealed_targets['row_id'] = df_revealed_targets['date_id'] + '_' + df_revealed_targets['seconds_in_bucket'] + '_' + df_revealed_targets['stock_id']
    df_revealed_targets['revealed_row_id'] = df_revealed_targets['revealed_date_id'] + '_' + df_revealed_targets['seconds_in_bucket'] + '_' + df_revealed_targets['stock_id']

    # USE_CONTINUOUS_UPDATEが有効の時、cacheのrow_idとdf_revealed_targetsのrevealed_row_idをleft joinする
    if USE_CONTINUOUS_UPDATE:
        df_r = df_revealed_targets[['revealed_row_id', 'revealed_target']]
        df_r.rename(columns={'revealed_target': 'target'}, inplace=True)
        df_r.rename(columns={'revealed_row_id': 'row_id'}, inplace=True)
        df_test = pd.merge(df_test, df_r, how='left', on='row_id')
        
    # USE_REVEALED_TARGETSが有効の時、cacheのrow_idとdf_revealed_targetsのrow_idをleft joinする
    if USE_REVEALED_TARGETS:
        df_r = df_revealed_targets[['row_id', 'revealed_target']]
        df_test = pd.merge(df_test, df_r, how='left', on='row_id')
        
    df_test = df_test.dropna(subset=["target"])
    df_test.reset_index(drop=True, inplace=True)
    return df_test

In [9]:
%%time
# Check if the code is running in offline or online mode
print("Load train dataset")

df_train = load_train_dataset()

if IS_MIN_LEARN:
    print("MIN LEARN MODE :", TARGET_STOCK_IDS)
    # In local mode, stock id TARGET_STOCK_ID is used for training
    df_train = df_train[df_train["stock_id"].isin(TARGET_STOCK_IDS)]
    
features = [c for c in df_train.columns if c not in ["row_id", "target", "time_id", "row_id", "date_id", "currently_scored"]]
print(features)

collect();
print(GetMemUsage())

Load train dataset
MIN LEARN MODE : [0]
['stock_id', 'seconds_in_bucket', 'imbalance_size', 'imbalance_buy_sell_flag', 'reference_price', 'matched_size', 'far_price', 'near_price', 'bid_price', 'bid_size', 'ask_price', 'ask_size', 'wap']
RAM memory GB usage = 1.398
CPU times: user 6.26 s, sys: 1.98 s, total: 8.23 s
Wall time: 9.09 s


In [10]:
%%time

print("Step1. Generate general Global Stock ID Features and basic features")
prev_cols = list(df_train.columns)
global_stock_id_feats = {
    "median_size": df_train.groupby("stock_id")["bid_size"].median() + df_train.groupby("stock_id")["ask_size"].median(),
    "std_size": df_train.groupby("stock_id")["bid_size"].std() + df_train.groupby("stock_id")["ask_size"].std(),
    "ptp_size": df_train.groupby("stock_id")["bid_size"].max() - df_train.groupby("stock_id")["bid_size"].min(),
    "median_price": df_train.groupby("stock_id")["bid_price"].median() + df_train.groupby("stock_id")["ask_price"].median(),
    "std_price": df_train.groupby("stock_id")["bid_price"].std() + df_train.groupby("stock_id")["ask_price"].std(),
    "ptp_price": df_train.groupby("stock_id")["bid_price"].max() - df_train.groupby("stock_id")["ask_price"].min(),
}

df_train = generate_basic_features(df_train)

generated_feature_name = list(set(df_train.columns) - set(prev_cols))
features += generated_feature_name
print(generated_feature_name)

collect()
print(GetMemUsage())

Step1. Generate general Global Stock ID Features and basic features
Memory usage of generate_basic_features is 22.00 MB
Memory usage after optimization is: 10.90 MB
Decreased by 50.46%
['spread_intensity', 'price_spread', 'all_sizes_skew', 'market_urgency', 'bid_size_diff_2', 'far_price_ask_price_imb', 'imbalance_size_shift_1', 'imbalance_size_shift_3', 'ask_size_diff_2', 'far_price_near_price_imb', 'bid_price_wap_imb', 'reference_price_shift_1', 'matched_imbalance', 'mid_price', 'matched_size_bid_size_ask_size_imb2', 'imbalance_buy_sell_flag_ret_2', 'reference_price_shift_2', 'ask_price_diff_10', 'ask_size_diff_3', 'reference_price_ret_10', 'matched_size_ask_size_imbalance_size_imb2', 'imbalance_size_ret_3', 'ask_price_bid_price_imb', 'size_imbalance', 'matched_size_ret_1', 'imbalance_size_ret_10', 'reference_price_wap_imb', 'matched_size_ret_10', 'imbalance_momentum', 'ask_price_diff_1', 'match_balance', 'ask_price_bid_price_reference_price_imb2', 'all_prices_skew', 'bid_price_diff_2

## Generation train features

In [11]:
%%time

print("Step2. Generate enhanced features")
prev_cols = list(df_train.columns)

def generate_historical_features(df):
    def cal_vix(df, group_key, target_col, period):
        return df.groupby(group_key)[target_col].transform(lambda x: np.log(x).diff().rolling(period).std())

    print("generate_historical_features")
    taget_cols = ['wap', 'match_balance']
    if USE_REVEALED_TARGETS:
        taget_cols.append('revealed_target')
    if USE_INDEX:
        taget_cols.append('index_mean_wap')
        taget_cols.append('index_std_wap')
        taget_cols.append('index_mean_match_balance')
        taget_cols.append('index_std_match_balance')
    for col in taget_cols:
        for window in [1, 3, 7]:
            col_name = f"{col}_diff_{window}"
            df[col_name] = df.groupby("stock_id")[col].diff(window)
            #df[col_name] = df[col_name].fillna(0)  # NaNを0で置き換える
        for period in [3, 5, 7]:
            col_name = f"{col}_vix_{period}"
            df[col_name] = cal_vix(df, ['stock_id', 'date_id'], col, period)
    df = df.replace([np.inf, -np.inf], 0)
    return df

def generate_index_features(df):
    print("generate_index_features")
     # Calculating mean and std for 'wap' and 'match_balance'
    wap_stats = df.groupby(['date_id', 'seconds_in_bucket'])['wap'].agg(['mean', 'std']).reset_index()
    match_balance_stats = df.groupby(['date_id', 'seconds_in_bucket'])['match_balance'].agg(['mean', 'std']).reset_index()

    # Adding prefix and suffix
    wap_stats = wap_stats.add_prefix('index_').add_suffix('_wap')
    match_balance_stats = match_balance_stats.add_prefix('index_').add_suffix('_match_balance')

    # Adjusting column names for merging
    wap_stats = wap_stats.rename(columns={'index_date_id_wap': 'date_id', 'index_seconds_in_bucket_wap': 'seconds_in_bucket'})
    match_balance_stats = match_balance_stats.rename(columns={'index_date_id_match_balance': 'date_id', 'index_seconds_in_bucket_match_balance': 'seconds_in_bucket'})

    # Merging with the original dataframe
    df = df.merge(wap_stats, on=['date_id', 'seconds_in_bucket'], how='left')
    df = df.merge(match_balance_stats, on=['date_id', 'seconds_in_bucket'], how='left')

    del wap_stats, match_balance_stats
    return df

def generate_enhance_features(df, is_train=False):
    if is_train:
        if USE_REVEALED_TARGETS:
            print("Use revealed targets")
            df[f"revealed_target"] = df.groupby(['date_id', 'stock_id'])['target'].shift(1)
        else:
            print("Dosent't use revealed targets")
    if USE_INDEX:
        print("Use index")
        df = generate_index_features(df)
    df = generate_historical_features(df)
    df = reduce_mem_usage(df, "generate_enhance_features")
    collect()
    return df

df_train = generate_enhance_features(df_train, is_train=True)
generated_feature_name = list(set(df_train.columns) - set(prev_cols))
features += generated_feature_name
print(generated_feature_name)

collect()
print(GetMemUsage())

Step2. Generate enhanced features
Dosent't use revealed targets
Use index
generate_index_features
generate_historical_features
Memory usage of generate_enhance_features is 16.55 MB
Memory usage after optimization is: 14.73 MB
Decreased by 10.98%
['index_mean_match_balance_diff_1', 'index_std_match_balance_diff_7', 'index_mean_match_balance_diff_3', 'wap_vix_7', 'index_mean_match_balance_vix_5', 'index_mean_match_balance_vix_3', 'index_std_wap_diff_3', 'match_balance_diff_1', 'wap_diff_1', 'match_balance_vix_7', 'index_std_wap_vix_5', 'index_mean_match_balance_vix_7', 'index_mean_wap_vix_3', 'index_mean_wap_diff_3', 'index_mean_wap_vix_7', 'index_mean_wap', 'match_balance_vix_5', 'match_balance_diff_3', 'index_mean_wap_diff_7', 'index_std_wap_diff_1', 'index_std_match_balance', 'index_mean_match_balance', 'match_balance_vix_3', 'index_std_wap_vix_3', 'wap_vix_5', 'index_std_wap', 'index_std_wap_vix_7', 'wap_vix_3', 'wap_diff_7', 'index_mean_wap_vix_5', 'index_std_match_balance_diff_3', 

In [12]:
cdf=None
if USE_REVEALED_TARGETS:
    cdf = df_train[['date_id', 'seconds_in_bucket', 'stock_id', 'target', 'revealed_target']].head(100)
cdf

In [13]:
%%time

"""
print("Step3. Normalized features")
prev_cols = list(df_train.columns)


# Global features for normarization
global_wap = df_train['wap'].describe()
global_mathch_balance = df_train['match_balance'].describe()
global_target = df_train['target'].describe()
global_reference_price = df_train['reference_price'].describe()

# 📈 Function to generate normalized features
def generate_normalized_features(df, is_train):
    print("generate_normalized_features")
    if is_train:
        df['n_target'] = (df['target'] - global_target['mean']) / global_target['std']
    df['n_wap'] = (df['wap'] - global_wap['mean']) / global_wap['std']
    df['n_match_balance'] = (df['match_balance'] - global_mathch_balance['mean']) / global_mathch_balance['std']
    df['n_reference_price'] = (df['reference_price'] - global_reference_price['mean']) / global_reference_price['std']
    
    df = reduce_mem_usage(df, "generate_normalized_features")
    return df

df_train = generate_normalized_features(df_train, True)
generated_feature_name = list(set(df_train.columns) - set(prev_cols))
generated_feature_name = [c for c in generated_feature_name if c not in ["n_target"]]

features += generated_feature_name
print(generated_feature_name)

collect();
print(GetMemUsage())
"""

CPU times: user 2 µs, sys: 1 µs, total: 3 µs
Wall time: 4.77 µs


'\nprint("Step3. Normalized features")\nprev_cols = list(df_train.columns)\n\n\n# Global features for normarization\nglobal_wap = df_train[\'wap\'].describe()\nglobal_mathch_balance = df_train[\'match_balance\'].describe()\nglobal_target = df_train[\'target\'].describe()\nglobal_reference_price = df_train[\'reference_price\'].describe()\n\n# 📈 Function to generate normalized features\ndef generate_normalized_features(df, is_train):\n    print("generate_normalized_features")\n    if is_train:\n        df[\'n_target\'] = (df[\'target\'] - global_target[\'mean\']) / global_target[\'std\']\n    df[\'n_wap\'] = (df[\'wap\'] - global_wap[\'mean\']) / global_wap[\'std\']\n    df[\'n_match_balance\'] = (df[\'match_balance\'] - global_mathch_balance[\'mean\']) / global_mathch_balance[\'std\']\n    df[\'n_reference_price\'] = (df[\'reference_price\'] - global_reference_price[\'mean\']) / global_reference_price[\'std\']\n    \n    df = reduce_mem_usage(df, "generate_normalized_features")\n    ret

In [14]:
df_train = reduce_mem_usage(df_train, 'train')

collect();
print(GetMemUsage())

Memory usage of train is 14.73 MB
Memory usage after optimization is: 14.73 MB
Decreased by 0.00%
RAM memory GB usage = 1.385


# Feature selection

In [15]:
# feature selection
if not USE_ALL_FEATUTES:
    features  = [
        "revealed_target",
        "wap_diff_1",
        "index_mean_wap_diff_1",
        "seconds_in_bucket",
        "stock_id",
    ]
#df_valid = df_train["target"]
#df_train = df_train[features]
if USE_REVEALED_TARGETS:
    features.remove("revealed_target")
features

['stock_id',
 'seconds_in_bucket',
 'imbalance_size',
 'imbalance_buy_sell_flag',
 'reference_price',
 'matched_size',
 'far_price',
 'near_price',
 'bid_price',
 'bid_size',
 'ask_price',
 'ask_size',
 'wap',
 'spread_intensity',
 'price_spread',
 'all_sizes_skew',
 'market_urgency',
 'bid_size_diff_2',
 'far_price_ask_price_imb',
 'imbalance_size_shift_1',
 'imbalance_size_shift_3',
 'ask_size_diff_2',
 'far_price_near_price_imb',
 'bid_price_wap_imb',
 'reference_price_shift_1',
 'matched_imbalance',
 'mid_price',
 'matched_size_bid_size_ask_size_imb2',
 'imbalance_buy_sell_flag_ret_2',
 'reference_price_shift_2',
 'ask_price_diff_10',
 'ask_size_diff_3',
 'reference_price_ret_10',
 'matched_size_ask_size_imbalance_size_imb2',
 'imbalance_size_ret_3',
 'ask_price_bid_price_imb',
 'size_imbalance',
 'matched_size_ret_1',
 'imbalance_size_ret_10',
 'reference_price_wap_imb',
 'matched_size_ret_10',
 'imbalance_momentum',
 'ask_price_diff_1',
 'match_balance',
 'ask_price_bid_price_ref

In [16]:
pd_display_max()
nan_count = df_train[features].isna().sum()
#df_train[features].to_csv('train.csv', index=False)
nan_count = nan_count[nan_count > 0].sort_values(ascending=False)
nan_count

index_std_match_balance_vix_5               26455
index_std_wap                               26455
index_std_match_balance_diff_7              26455
index_std_wap_diff_3                        26455
index_std_wap_vix_5                         26455
index_std_wap_diff_1                        26455
index_std_match_balance                     26455
index_std_wap_vix_3                         26455
index_std_wap_vix_7                         26455
index_std_match_balance_diff_3              26455
index_std_wap_diff_7                        26455
index_std_match_balance_vix_3               26455
index_std_match_balance_diff_1              26455
index_std_match_balance_vix_7               26455
depth_pressure                              14517
far_price_wap_imb                           14517
far_price_bid_price_imb                     14517
reference_price_far_price_imb               14517
far_price                                   14517
far_price_ask_price_imb                     14517


In [17]:
pd_clear_display_max()

# Train function (lightgbm)

In [18]:
# 📦 Import necessary libraries
import numpy as np
import lightgbm as lgb
from sklearn.metrics import mean_absolute_error
import gc
import os
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import KFold
import numpy as np
from dataclasses import dataclass
import sys
import shutil
import lightgbm as lgb

from warnings import simplefilter
simplefilter("ignore", category=RuntimeWarning)

@dataclass
class Model:
    booster: lgb.Booster
    fold: int
    feature_importance: pd.DataFrame
    score: float
    best_iteration: int
    train_time: float = None
    weight: float = None
    mem_usage: float = None
    train_func: str = None
    is_latest: bool = False

def train_model(train_x, train_y, val_x, val_y, best_params=None):
    trains = lgb.Dataset(train_x, train_y)
    valids = lgb.Dataset(val_x, val_y, reference=trains)

    verbose_eval = -1
    if best_params is None:
        params = lgb_params
    else:
        params = best_params

    print("Use params:")
    print(params)

    booster = lgb.train(
        params,
        trains,
        valid_sets=valids, # 検証データ
        num_boost_round=num_boost_round,
        keep_training_booster=True,
        callbacks=[
                lgb.early_stopping(stopping_rounds=stopping_rounds, verbose=True),
                lgb.log_evaluation(verbose_eval)
        ]
    )

    del trains, valids
    return booster

def cross_train(df, key, n_splits, features, valid_name, best_params=None):
    """ For Cross Train

    Args:
        df (_type_): _description_
        n_splits (_type_): _description_

    Returns:
        _type_: _description_
    """
    print("----------------------------------------")
    print(f"Cross Train key id {key}: start, shape: {df_train.shape}, n_splits: {n_splits}")
    print(f"num_boost_round: {num_boost_round}, stopping_rounds: {stopping_rounds}, folds: {num_folds}")

    models = []
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)
    df.reset_index(drop=True, inplace=True)
    
    for fold, (train_indices, valid_indices) in enumerate(kf.split(df)):
        print(f"----- Train {key}: {fold} start -----")
        now_time = time.time()

        print(f"train_indices: {train_indices}, valid_indices: {valid_indices}")
        X_train, X_valid = df[features].iloc[train_indices], df[features].iloc[valid_indices]
        y_train, y_valid = df[valid_name].loc[train_indices], df[valid_name].loc[valid_indices]
        print(f"X_train: {X_train.shape}, X_valid: {X_valid.shape}, y_train: {y_train.shape}, y_valid: {y_valid.shape}")

        booster = train_model(X_train, y_train, X_valid, y_valid, best_params)
        print(booster.best_score)
        y_valid_pred = booster.predict(X_valid)
        
        score = mean_absolute_error(y_valid, y_valid_pred)
        train_time = time.time() - now_time
        mem_usage = sys.getsizeof(booster) / (1024 * 1024) # MB
        model = Model(booster, fold, booster.feature_importance(), score, booster.best_iteration, train_time, weight= 1 / n_splits, mem_usage=mem_usage, train_func="lightgbm", is_latest=True)
        print(f"{key}: {fold} end, score: {score}, time: {model.train_time}, best_iteration: {model.best_iteration}, memory usage: {model.mem_usage}")
        
        models.append(model)
        
        del X_train, X_valid, y_train, y_valid
        gc.collect()
        print(GetMemUsage())
        print(f"----- Train {key}: {fold} end -----")

    print(f"Cross train {key} model len {len(models)}")
    print("----------------------------------------")
    return key, models

# Train function (optuna)

In [19]:
%%time

import optuna.integration.lightgbm as optuna_lgb
import optuna
import lightgbm
optuna.logging.set_verbosity(optuna.logging.ERROR)

class TunerCVCheckpointCallback(object):
    """Optuna の LightGBMTunerCV から学習済みモデルを取り出すためのコールバック"""

    def __init__(self):
        # Models
        self.models = []
        self.counter = 0

    def get_models(self):
        # https://lightgbm.readthedocs.io/en/latest/pythonapi/lightgbm.Booster.html#lightgbm.Booster
        return self.models

    def __call__(self, env: lightgbm.callback.CallbackEnv):
        """_summary_

        Args:
            env (lightgbm.callback.CallbackEnv): _description_
            "model",
            "params",
            "iteration",
            "begin_iteration",
            "end_iteration",
            "evaluation_result_list"
        """
        print("")

        self.counter += 1
        print("-------------------")
        print(f"Counter: {self.counter}")
        print(f"Iteration: {env.iteration}")
        print(f"Begin_iteration: {env.begin_iteration}")
        print(f"End_iteration: {env.end_iteration}")
        print(f"Evaluation_result_list: {env.evaluation_result_list}")
        print(f"Model best_iteration: {env.model.best_iteration}")
        print("Params: ", env.params)
        #self.models.append(env.model)
        del env

        collect();
        print(GetMemUsage())

def optuna_tuning(df, n_splits, features, valid_name, model_save_path):
    df_train = df[features]
    df_valid = df[valid_name]
    
    trains = optuna_lgb.Dataset(df_train, df_valid)
    
    print("------- Optuna Tuning Start -------")
    now_time = time.time()
    print(f"num_boost_round: {num_boost_round}, stopping_rounds: {stopping_rounds}, folds: {num_folds}")

    folds = KFold(n_splits=n_splits, shuffle=True, random_state=42)
    checkpoint_cb = TunerCVCheckpointCallback()
    
    verbose_eval = 0
    # https://optuna.readthedocs.io/en/stable/reference/generated/optuna.integration.lightgbm.LightGBMTunerCV.html
    tuner = optuna_lgb.LightGBMTunerCV(
        optuna_params,
        trains,
        num_boost_round=num_boost_round,
        folds=folds,
        show_progress_bar=False,
        return_cvbooster=True,
        verbosity=-1,
        model_dir=model_save_path,
        optuna_seed=seed,
        time_budget=OPTUNA_TIME_BUDGET,
        callbacks=[
                lgb.early_stopping(stopping_rounds=stopping_rounds, verbose=True),
                lgb.log_evaluation(verbose_eval),
                checkpoint_cb
        ]
    )
    
    tuner.run()
    best_params = tuner.best_params
    
    print("Params: ")
    for key, value in best_params.items():
        print(" {}: {}".format(key, value))

    print("")
    print("len(tuner.study.trials): ", len(tuner.study.trials))
    #print("len(checkpoint_cb.cv_boosters): ", len(checkpoint_cb.models))
    print("Tuner best_params", tuner.best_params)
    print("Tuner best score: ", tuner.best_score)
   
    # 最も良かったパラメータをキーにして学習済みモデルを取り出す
    best_booster = tuner.get_best_booster()
    score = -1
    train_time = time.time() - now_time
    mem_usage = sys.getsizeof(best_booster) / (1024 * 1024) # MB
    feature_importance = np.mean(best_booster.feature_importance(), axis=0)

    best_model = Model(best_booster, 1, feature_importance, score, best_booster.best_iteration, train_time, weight= 1, mem_usage=mem_usage, train_func="optuna_lgb")
    print("------- Optuna Tuning End -------")
    return best_params, best_model


CPU times: user 163 ms, sys: 27.4 ms, total: 190 ms
Wall time: 741 ms


# Training

In [20]:
%%time

KEY = "-1"

# Train
best_params = None
key_models = None
if USE_OPTUNA:
    model_save_base_path = f"{BASE_OUTPUT_PATH}/model"
    if os.path.exists(model_save_base_path):
        print(f"{model_save_base_path} already exists, clean up it.")
        shutil.rmtree(model_save_base_path)
    os.makedirs(model_save_base_path)
    print(f"model_save_base_path: {model_save_base_path}")

    best_params, best_model = optuna_tuning(df=df_train, n_splits=num_folds, features=features, valid_name="target", model_save_path=model_save_base_path)
    key_models = [(KEY, [best_model])]
else:
    #key_models = df_train.groupby("seconds_in_bucket").apply(lambda x: cross_train(df=x, key=x.name, n_splits=num_folds, feature_name=feature_name, valid_name="target", best_params=best_params))
    key_models = [cross_train(df_train, key=KEY, n_splits=num_folds, features=features, valid_name="target", best_params=best_params)]
    if IS_USE_SAVED_MODEL:
        model_save_base_path = f"{BASE_OUTPUT_PATH}/model"
        if os.path.exists(model_save_base_path):
            print(f"{model_save_base_path} already exists, clean up it.")
            shutil.rmtree(model_save_base_path)
        os.makedirs(model_save_base_path)

        key_model_paths = []
        for key, models in key_models:
            model_save_path = f"{model_save_base_path}/{key}"
            os.makedirs(model_save_path)
            model_paths = []
            for model in models:
                model_save_fullpath = f"{model_save_path}/model_{key}_{model.fold}.txt"
                model.model.save_model(model_save_fullpath)
                model_paths.append(model_save_fullpath)
            key_model_paths.append((key, model_paths))

        model_dict_saved = {key: model_paths for key, model_paths in key_model_paths}
        print(model_dict_saved)


model_dict = {key: model for key, model in key_models}
collect()
print(GetMemUsage())

----------------------------------------
Cross Train key id -1: start, shape: (26455, 148), n_splits: 2
num_boost_round: 1, stopping_rounds: 1, folds: 2
----- Train -1: 0 start -----
       stock_id  date_id  seconds_in_bucket  imbalance_size  \
0             0        0                  0    3.180603e+06   
1             0        0                 10    1.299773e+06   
2             0        0                 20    1.299773e+06   
3             0        0                 30    1.299773e+06   
4             0        0                 40    1.218204e+06   
...         ...      ...                ...             ...   
26450         0      480                500    0.000000e+00   
26451         0      480                510    0.000000e+00   
26452         0      480                520    4.755137e+05   
26453         0      480                530    4.755137e+05   
26454         0      480                540    4.755137e+05   

       imbalance_buy_sell_flag  ...  index_std_match_balance

# Update model using test

In [21]:
%%time

# global train cache for continuous update
global_train_cache = df_train.copy()
# origin 0 is train, 1 is test, 2 is revaled
global_train_cache['origin'] = 0
date_duration = DATA_COUNT_IN_SAME_BUCKET * continuos_dataset_span
print("date_duration", date_duration)

def update_global_train_cache(df, origin, valid_key: str = 'target'):
    global global_train_cache
    df['origin'] = origin
    print("update_global_train_cache")
    global_train_cache = pd.concat([global_train_cache, df], axis=0)
    global_train_cache = global_train_cache.dropna(subset=['target'])
    global_train_cache = global_train_cache.sort_values(by=['date_id', 'seconds_in_bucket', 'stock_id', 'origin'])
    global_train_cache = global_train_cache.drop_duplicates(['date_id', 'seconds_in_bucket', 'stock_id'], keep='last')
    global_train_cache.reset_index(drop=True, inplace=True)
    global_train_cache = global_train_cache.groupby(['stock_id']).tail(date_duration).sort_values(by=['date_id', 'seconds_in_bucket', 'stock_id']).reset_index(drop=True)

    global_train_cache = reduce_mem_usage(global_train_cache, 'global_train_cache')
    print(f"Updated global_train_cache, len: ", len(global_train_cache))
    if IS_DEBUG:
        print(len(global_train_cache))
        if USE_REVEALED_TARGETS:
            cdf = global_train_cache[['date_id', 'seconds_in_bucket', 'stock_id', 'origin', 'revealed_target', 'target']]
        else:
            cdf = global_train_cache[['date_id', 'seconds_in_bucket', 'stock_id', 'origin', 'target']]
        print(cdf)

def update_models(df, models, features, valid_name):
    """ For Update Model

    Args:
        df (_type_): _description_
        n_splits (_type_): _description_

    Returns:
        _type_: _description_
    """

    kf = KFold(n_splits=num_folds, shuffle=True, random_state=42)
    df.reset_index(drop=True, inplace=True)
    
    for fold, (train_indices, valid_indices) in enumerate(kf.split(df)):
        print(f"{key}: {fold} update")
        now_time = time.time()
        X_train, X_valid = df[features].iloc[train_indices], df[features].iloc[valid_indices]
        y_train, y_valid = df[valid_name].loc[train_indices], df[valid_name].loc[valid_indices]
        trains = lgb.Dataset(X_train, y_train, free_raw_data=False)
        valids = lgb.Dataset(X_valid, y_valid, reference=trains)

        print(f"X_train: {X_train.shape}, X_valid: {X_valid.shape}, y_train: {y_train.shape}, y_valid: {y_valid.shape}")
        models[fold].booster.update(trains)

def keep_train_models(df, models, features, valid_name, is_append=False):
    print(f"----------------- keep_train_models, is_append: {is_append}, model len: {len(models)}, df len: {len(df)} ---------------------")
    train_x = df[features]
    train_y = df[valid_name]
    trains = lgb.Dataset(train_x, train_y, free_raw_data=False)
    verbose_eval = -1

    counter = 0
    r_models = []
    if IS_DEBUG:
        print(f"Re-train dataset:")
        if USE_REVEALED_TARGETS:
            print(df[['date_id', 'seconds_in_bucket', 'stock_id', 'target', 'revealed_target']])
        else:
            print(df[['date_id', 'seconds_in_bucket', 'stock_id', 'target']])
    for model in models:
        print(f"---- train start, counter: {counter} ----")
        if model.is_latest:
            print("Update latest model")
            now_time = time.time()
            booster = lgb.train(
                lgb_params,
                trains,
                num_boost_round=update_num_boost_round,
                keep_training_booster=True,
                init_model=model.booster,
            )
            train_time = time.time() - now_time
            updated_model = Model(
                booster=booster,
                fold=1,
                best_iteration=booster.best_iteration, 
                feature_importance=booster.feature_importance(),
                score=-1, 
                train_time=train_time, 
                weight=-1, 
                mem_usage=-1,
                is_latest=True,
                train_func="lightgbm update by test")
            r_models.append(updated_model)
            if is_append:
                print("Adding previous model")
                model.is_latest = False
                r_models.append(model)
        else:
            print("Dose not latest, just append")
            r_models.append(model)
        counter = counter + 1
    print(f"---- train end, train time: {train_time}, updated model len {len(r_models)} ----")
    return r_models

if USE_CONTINUOUS_UPDATE:
    try:
        print("Update model with test date")
        df_test = load_test_dataset()
        if IS_MIN_LEARN:
            print("MIN LEARN MODE :", TARGET_STOCK_IDS)
            df_test = df_test[df_test["stock_id"].isin(TARGET_STOCK_IDS)]
        df_test = generate_basic_features(df_test)
        df_test = generate_enhance_features(df_test)
        update_global_train_cache(df_test, 1)
        #model_dict[KEY] = keep_train_models(global_train_cache, model_dict[KEY], features, "target", is_append=True)
        print("Update model with test date end")
    except Exception as e:
        print("Cannot get test date", e)

collect()
print(GetMemUsage())


date_duration 165
Update model with test date
MIN LEARN MODE : [0]
Memory usage of generate_basic_features is 0.09 MB
Memory usage after optimization is: 0.05 MB
Decreased by 50.46%
Use index
generate_index_features
generate_historical_features
Memory usage of generate_enhance_features is 0.07 MB
Memory usage after optimization is: 0.06 MB
Decreased by 10.96%
update_global_train_cache
Memory usage of global_train_cache is 0.09 MB
Memory usage after optimization is: 0.09 MB
Decreased by 1.18%
Updated global_train_cache, len:  165
165
     date_id  seconds_in_bucket  stock_id  origin    target
0        478                  0         0       1 -5.429983
1        478                 10         0       1 -3.489852
2        478                 20         0       1 -3.330112
3        478                 30         0       1 -1.900196
4        478                 40         0       1  2.189875
..       ...                ...       ...     ...       ...
160      480                500         0

In [22]:
# Show results

for key, models in model_dict.items():
    print(f"Key: {key}, model len: {len(models)}")
    data = []
    for model in models:
        score = model.score
        best_iteration = model.best_iteration
        fold = model.fold
        train_time = model.train_time
        data.append({"key": key, "fold": fold, "score": score, "best_iteration": best_iteration, "train_time": train_time})

    df_model = pd.DataFrame(data)
    print(df_model.describe())

Key: -1, model len: 2
           fold     score  best_iteration  train_time
count  2.000000  2.000000             2.0    2.000000
mean   0.500000  4.379504             1.0    0.221114
std    0.707107  0.023088             0.0    0.028631
min    0.000000  4.363178             1.0    0.200868
25%    0.250000  4.371341             1.0    0.210991
50%    0.500000  4.379504             1.0    0.221114
75%    0.750000  4.387667             1.0    0.231236
max    1.000000  4.395830             1.0    0.241359


In [23]:
# Check model quality
data = []

for key, i_models in model_dict.items():
    for model in i_models:
        score = model.score
        best_iteration = model.best_iteration
        fold = model.fold
        train_time = model.train_time
        data.append({"key": key, "fold": fold, "score": score, "best_iteration": best_iteration, "train_time": train_time})

df_model = pd.DataFrame(data)
df_model.describe()

,fold,score,best_iteration,train_time
count,2.000000,2.000000,2.0,2.000000
mean,0.500000,4.379504,1.0,0.221114
std,0.707107,0.023088,0.0,0.028631
min,0.000000,4.363178,1.0,0.200868
25%,0.250000,4.371341,1.0,0.210991
50%,0.500000,4.379504,1.0,0.221114
75%,0.750000,4.387667,1.0,0.231236
max,1.000000,4.395830,1.0,0.241359


In [24]:
# Initialize an empty DataFrame for aggregated importances
aggregated_importance = pd.DataFrame(index=features, columns=['importance'])

# Aggregate the importances from each model
for key, i_models in model_dict.items():
    for model in i_models:
        importance = pd.DataFrame({'feature': features, 'importance': model.feature_importance})
        aggregated_importance = aggregated_importance.add(importance.set_index('feature'), fill_value=0)

aggregated_importance['importance'] /= len(df_model)

pd_display_max()
# Sort the features by importance
aggregated_importance = aggregated_importance.sort_values(by='importance', ascending=False)
aggregated_importance

,importance
reference_price_wap_imb,2.0
imbalance_size,1.0
imbalance_size_shift_10,1.0
matched_size_bid_size_imbalance_size_imb2,0.5
matched_size_ret_2,0.5
match_balance_vix_7,0.5
reference_price_shift_1,0.5
price_spread,0.5
spread_intensity,0.5
reference_price_ret_3,0.5


# Dataset types

In [25]:
features_types = df_train[features].dtypes
features_types

stock_id                                        int8
seconds_in_bucket                              int16
imbalance_size                               float32
imbalance_buy_sell_flag                         int8
reference_price                              float32
matched_size                                 float32
far_price                                    float32
near_price                                   float32
bid_price                                    float32
bid_size                                     float32
ask_price                                    float32
ask_size                                     float32
wap                                          float32
spread_intensity                             float32
price_spread                                 float32
all_sizes_skew                               float32
market_urgency                               float32
bid_size_diff_2                              float32
far_price_ask_price_imb                      f

In [26]:
def convert_dtypes(df):
    df_types = df[features].dtypes
    different_types = [col for col in df_types.index if col in features_types and df_types[col] != features_types[col]]
    print(f"Different Types: {different_types}")
    return different_types

def update_dtypes_by_origin(df):
    diff_types = convert_dtypes(df)
    for col in diff_types:
        df[col] = df[col].astype(features_types[col])
    return df

# Clear trains

In [27]:
# Clean up
pd_clear_display_max()
del key_models
if IS_USE_SAVED_MODEL:
    print("Delete model_dict")
    del model_dict
# del df_train
collect()
print(GetMemUsage())

RAM memory GB usage = 1.501


# Infer

In [28]:
%%time

y_min, y_max = -64, 64
predictions = []
cache = pd.DataFrame()
df_result = pd.DataFrame()

counter = 0
df_revealed_targets = pd.DataFrame()

if IS_INFER:
    if IS_LOCAL or IS_DEBUG:
        print("Infer Local")
        env = make_env()
    else:
        print("Infer Submission")
        import optiver2023
        env = optiver2023.make_env()
    iter_test = env.iter_test()

    try:
        for (test, revealed_targets, sample_prediction) in iter_test:
            now_time = time.time()
            print(f"------- counter {counter} start -------")

            # It faults due to test is iterator
            #seconds_in_bucket = test['seconds_in_bucket'][0]
            #print(f"prdict: {test['date_id'][0]}, {seconds_in_bucket}")

            # Generate cahce
            cache = pd.concat([cache, test], ignore_index=True, axis=0)
            if IS_MIN_LEARN:
                print("MIN LEARN MODE :", TARGET_STOCK_IDS)
                cache = cache[cache["stock_id"].isin(TARGET_STOCK_IDS)]
            if counter > 0:
                # Clear cache data
                cache = cache.groupby(['stock_id']).tail(DATA_COUNT_IN_SAME_BUCKET * continuos_train_span + 1).sort_values(by=['date_id', 'seconds_in_bucket', 'stock_id']).reset_index(drop=True)
                if USE_CONTINUOUS_UPDATE:
                    cache = cache.drop(['target'], axis=1)
                if USE_REVEALED_TARGETS:
                    cache = cache.drop(['revealed_target'], axis=1)
            print(f"cache len {len(cache)}")

            # Add revealed target as target for counituous update
            copy_revealed_targets = revealed_targets.copy()
            copy_revealed_targets = copy_revealed_targets.dropna()
            print("copy_revealed_targets len", len(copy_revealed_targets))

            if len(copy_revealed_targets) > 0:
                print("Update revealed_targets")
                copy_revealed_targets['revealed_date_id'] = copy_revealed_targets['revealed_date_id'].astype(int).astype(str)
                copy_revealed_targets['date_id'] = copy_revealed_targets['date_id'].astype(int).astype(str)
                copy_revealed_targets['seconds_in_bucket'] = copy_revealed_targets['seconds_in_bucket'].astype(int).astype(str)
                copy_revealed_targets['stock_id'] = copy_revealed_targets['stock_id'].astype(int).astype(str)  # Converting to int first to remove any decimal points
                copy_revealed_targets['revealed_row_id'] = copy_revealed_targets['revealed_date_id'] + '_' + copy_revealed_targets['seconds_in_bucket'] + '_' + copy_revealed_targets['stock_id']
                copy_revealed_targets['row_id'] = copy_revealed_targets['date_id'] + '_' + copy_revealed_targets['seconds_in_bucket'] + '_' + copy_revealed_targets['stock_id']
                copy_revealed_targets['revealed_target'] = copy_revealed_targets['revealed_target'].astype('float32')

                df_revealed_targets = pd.concat([df_revealed_targets, copy_revealed_targets], ignore_index=True, axis=0)
                # 余分なデータを削除する、範囲はcacheの2倍必要
                df_revealed_targets = df_revealed_targets.groupby(['stock_id']).tail((DATA_COUNT_IN_SAME_BUCKET * continuos_train_span + 1) * 3).sort_values(by=['date_id', 'seconds_in_bucket', 'stock_id']).reset_index(drop=True)
                df_revealed_targets = reduce_mem_usage(df_revealed_targets, 'df_revealed_targets')

            # USE_CONTINUOUS_UPDATEが有効の時、cacheのrow_idとdf_revealed_targetsのrevealed_row_idをleft joinする
            if USE_CONTINUOUS_UPDATE:
                df_r = df_revealed_targets[['revealed_row_id', 'revealed_target']]
                df_r.rename(columns={'revealed_target': 'target'}, inplace=True)
                df_r.rename(columns={'revealed_row_id': 'row_id'}, inplace=True)
                cache = pd.merge(cache, df_r, how='left', on='row_id')


            # USE_REVEALED_TARGETSが有効の時、cacheのrow_idとdf_revealed_targetsのrow_idをleft joinする
            if USE_REVEALED_TARGETS:
                df_r = df_revealed_targets[['row_id', 'revealed_target']]
                cache = pd.merge(cache, df_r, how='left', on='row_id')

            cache = reduce_mem_usage(cache, 'cache')

            # Generate features
            df_valid = generate_basic_features(cache)
            df_valid = generate_enhance_features(df_valid)
            df_valid = reduce_mem_usage(df_valid, 'df_valid')

            # Update model
            if (counter > 0) and (counter % (DATA_COUNT_IN_SAME_BUCKET * continuos_train_span) == 0) and USE_CONTINUOUS_UPDATE:
                train_now_time = time.time()
                print("Update model with revealed_target date start")
                print(f"df_valid len {len(df_valid)}")
                df_train = df_valid.copy()
                df_train.dropna(subset=['target'], inplace=True)
                df_train = update_dtypes_by_origin(df_train)
                df_train.reset_index(drop=True, inplace=True)

                update_global_train_cache(df_train, 2)
                model_dict[KEY] = keep_train_models(global_train_cache, model_dict[KEY], features, "target", True)
                print(f"ReTrain Time: {time.time() - train_now_time}")
                del df_train

            if not IS_MIN_LEARN:
                # stockの数分取得する
                df_valid = df_valid[-len(test):].reset_index(drop=True)
            else:
                df_valid = df_valid[-len(TARGET_STOCK_IDS):].reset_index(drop=True)

            # Get seconds_in_bucket and date_id
            seconds_in_bucket = df_test['seconds_in_bucket'][0]
            date_id = df_test['date_id'][0]
            print(f"Predict: {date_id}, {seconds_in_bucket}")

            # Predict
            predictions = model_infer(KEY, df_valid[features])
            df_valid['pred'] = predictions
            scaled_predictions = zero_clip(df_valid, predictions)
            df_valid['scaled_pred'] = scaled_predictions

            # For save
            if IS_DEBUG:
                df_result = pd.concat([df_result, df_valid], ignore_index=True, axis=0)

            # Submit
            if not IS_MIN_LEARN:
                print("Submit prediction")
                sample_prediction['target'] = scaled_predictions
                env.predict(sample_prediction)
            else:
                print("Submit dummy prediction")
                sample_prediction['target'] = 0
                env.predict(sample_prediction)

            # Clean up
            execution_time = time.time() - now_time
            del df_valid, predictions, scaled_predictions
            collect()
            print(GetMemUsage())
            print(f"------- counter {counter}, execution_time {execution_time} end -------")
            counter += 1
    except Exception as e:
        print("Error", e)

collect()
print(GetMemUsage())
if IS_DEBUG:
    df_result.to_csv(f"{BASE_OUTPUT_PATH}/result.csv", index=False)

CPU times: user 7 µs, sys: 1e+03 ns, total: 8 µs
Wall time: 11 µs


In [29]:
%%time

y_min, y_max = -64, 64
predictions = []
cache = pd.DataFrame()
df_result = pd.DataFrame()

counter = 0
df_revealed_targets = pd.DataFrame()

if IS_INFER:
    if IS_LOCAL:
        print("Infer Local")
        env = make_env()
    else:
        print("Infer Submission")
        import optiver2023
        env = optiver2023.make_env()
    iter_test = env.iter_test()

    for (test, revealed_targets, sample_prediction) in iter_test:
        now_time = time.time()
        print(f"------- counter {counter} start -------")

        # It faults due to test is iterator
        #seconds_in_bucket = test['seconds_in_bucket'][0]
        #print(f"prdict: {test['date_id'][0]}, {seconds_in_bucket}")

        # Generate cahce
        cache = pd.concat([cache, test], ignore_index=True, axis=0)
        if IS_MIN_LEARN:
            print("MIN LEARN MODE :", TARGET_STOCK_IDS)
            cache = cache[cache["stock_id"].isin(TARGET_STOCK_IDS)]
        if counter > 0:
            # Clear cache data
            cache = cache.groupby(['stock_id']).tail(DATA_COUNT_IN_SAME_BUCKET * continuos_train_span + 1).sort_values(by=['date_id', 'seconds_in_bucket', 'stock_id']).reset_index(drop=True)
            if USE_CONTINUOUS_UPDATE:
                cache = cache.drop(['target'], axis=1)
            if USE_REVEALED_TARGETS:
                cache = cache.drop(['revealed_target'], axis=1)
        print(f"cache len {len(cache)}")

        # Add revealed target as target for counituous update
        copy_revealed_targets = revealed_targets.copy()
        copy_revealed_targets = copy_revealed_targets.dropna()
        print("copy_revealed_targets len", len(copy_revealed_targets))
        
        if len(copy_revealed_targets) > 0:
            print("Update revealed_targets")
            copy_revealed_targets['revealed_date_id'] = copy_revealed_targets['revealed_date_id'].astype(int).astype(str)
            copy_revealed_targets['date_id'] = copy_revealed_targets['date_id'].astype(int).astype(str)
            copy_revealed_targets['seconds_in_bucket'] = copy_revealed_targets['seconds_in_bucket'].astype(int).astype(str)
            copy_revealed_targets['stock_id'] = copy_revealed_targets['stock_id'].astype(int).astype(str)  # Converting to int first to remove any decimal points
            copy_revealed_targets['revealed_row_id'] = copy_revealed_targets['revealed_date_id'] + '_' + copy_revealed_targets['seconds_in_bucket'] + '_' + copy_revealed_targets['stock_id']
            copy_revealed_targets['row_id'] = copy_revealed_targets['date_id'] + '_' + copy_revealed_targets['seconds_in_bucket'] + '_' + copy_revealed_targets['stock_id']
            copy_revealed_targets['revealed_target'] = copy_revealed_targets['revealed_target'].astype('float32')

            df_revealed_targets = pd.concat([df_revealed_targets, copy_revealed_targets], ignore_index=True, axis=0)
            # 余分なデータを削除する、範囲はcacheの2倍必要
            df_revealed_targets = df_revealed_targets.groupby(['stock_id']).tail((DATA_COUNT_IN_SAME_BUCKET * continuos_train_span + 1) * 3).sort_values(by=['date_id', 'seconds_in_bucket', 'stock_id']).reset_index(drop=True)
            df_revealed_targets = reduce_mem_usage(df_revealed_targets, 'df_revealed_targets')

        # USE_CONTINUOUS_UPDATEが有効の時、cacheのrow_idとdf_revealed_targetsのrevealed_row_idをleft joinする
        if USE_CONTINUOUS_UPDATE:
            df_r = df_revealed_targets[['revealed_row_id', 'revealed_target']]
            df_r.rename(columns={'revealed_target': 'target'}, inplace=True)
            df_r.rename(columns={'revealed_row_id': 'row_id'}, inplace=True)
            cache = pd.merge(cache, df_r, how='left', on='row_id')

            
        # USE_REVEALED_TARGETSが有効の時、cacheのrow_idとdf_revealed_targetsのrow_idをleft joinする
        if USE_REVEALED_TARGETS:
            df_r = df_revealed_targets[['row_id', 'revealed_target']]
            cache = pd.merge(cache, df_r, how='left', on='row_id')

        cache = reduce_mem_usage(cache, 'cache')

        # Generate features
        df_valid = generate_basic_features(cache)
        df_valid = generate_enhance_features(df_valid)
        df_valid = reduce_mem_usage(df_valid, 'df_valid')

        # Update model
        if (counter > 0) and (counter % (DATA_COUNT_IN_SAME_BUCKET * continuos_train_span) == 0) and USE_CONTINUOUS_UPDATE:
            train_now_time = time.time()
            print("Update model with revealed_target date start")
            print(f"df_valid len {len(df_valid)}")
            df_train = df_valid.copy()
            df_train.dropna(subset=['target'], inplace=True)
            df_train = update_dtypes_by_origin(df_train)
            df_train.reset_index(drop=True, inplace=True)

            update_global_train_cache(df_train, 2)
            model_dict[KEY] = keep_train_models(global_train_cache, model_dict[KEY], features, "target", True)
            print(f"ReTrain Time: {time.time() - train_now_time}")
            del df_train

        if not IS_MIN_LEARN:
            # stockの数分取得する
            df_valid = df_valid[-len(test):].reset_index(drop=True)
        else:
            df_valid = df_valid[-len(TARGET_STOCK_IDS):].reset_index(drop=True)

        # Get seconds_in_bucket and date_id
        seconds_in_bucket = df_test['seconds_in_bucket'][0]
        date_id = df_test['date_id'][0]
        print(f"Predict: {date_id}, {seconds_in_bucket}")
        
        # Predict
        predictions = model_infer(KEY, df_valid[features])
        df_valid['pred'] = predictions
        scaled_predictions = zero_clip(df_valid, predictions)
        df_valid['scaled_pred'] = scaled_predictions

        # For save
        if IS_DEBUG:
            df_result = pd.concat([df_result, df_valid], ignore_index=True, axis=0)
            
        # Submit
        if not IS_MIN_LEARN:
            print("Submit prediction")
            sample_prediction['target'] = scaled_predictions
            env.predict(sample_prediction)
        else:
            print("Submit dummy prediction")
            sample_prediction['target'] = 0
            env.predict(sample_prediction)

        # Clean up
        execution_time = time.time() - now_time
        print(f"------- counter {counter}, execution_time {execution_time} end -------")
        del df_valid, predictions, scaled_predictions
        collect()
        counter += 1

collect()
print(GetMemUsage())
if IS_DEBUG:
    df_result.to_csv(f"{BASE_OUTPUT_PATH}/result.csv", index=False)

Infer Local
------- counter 0 start -------
MIN LEARN MODE : [0]
cache len 1
copy_revealed_targets len 11000
Update revealed_targets
Memory usage of df_revealed_targets is 0.71 MB
Memory usage after optimization is: 0.61 MB
Decreased by 14.70%
Memory usage of cache is 0.00 MB
Memory usage after optimization is: 0.00 MB
Decreased by 27.65%
Memory usage of generate_basic_features is 0.00 MB
Memory usage after optimization is: 0.00 MB
Decreased by 10.34%
Use index
generate_index_features
generate_historical_features
Memory usage of generate_enhance_features is 0.00 MB
Memory usage after optimization is: 0.00 MB
Decreased by 9.15%
Memory usage of df_valid is 0.00 MB
Memory usage after optimization is: 0.00 MB
Decreased by 0.00%
Predict: 478, 0
Predictor target models len 2
std_predictions [0.05417345]
Submit dummy prediction
------- counter 0, execution_time 1.053267002105713 end -------
------- counter 1 start -------
MIN LEARN MODE : [0]
cache len 2
copy_revealed_targets len 0
Memory usa

KeyboardInterrupt: 

In [ ]:
if IS_DEBUG:
    df_revealed_targets = pd.read_csv(REVEALED_TARGETS_FILE)
    df_revealed_targets = df_revealed_targets.dropna(subset=['revealed_date_id', 'seconds_in_bucket', 'stock_id'])
    df_revealed_targets['revealed_date_id'] = df_revealed_targets['revealed_date_id'].astype(int).astype(str)
    df_revealed_targets['seconds_in_bucket'] = df_revealed_targets['seconds_in_bucket'].astype(int).astype(str)
    df_revealed_targets['stock_id'] = df_revealed_targets['stock_id'].astype(int).astype(str)  # Converting to int first to remove any decimal points

    # Concatenate the columns
    df_revealed_targets['row_id'] = df_revealed_targets['revealed_date_id'] + '_' + df_revealed_targets['seconds_in_bucket'] + '_' + df_revealed_targets['stock_id']
    
    df_pred = df_result[['row_id', 'scaled_pred']]
    df = pd.merge(df_pred, df_revealed_targets, how='left', on='row_id')

    df = df.rename(columns={'revealed_target': 'target'})
    df = df.dropna(subset=['target'])
    #df['score'] = (df['scaled_pred'] - df['target']).abs()

    df['score'] = mean_absolute_error(df['scaled_pred'], df['target'])
    df = df[['row_id', 'scaled_pred', 'target', 'score']]
    print(df.describe())

    print("  ")
    print(df)